## Import taxi_clean

In [11]:
import pandas as pd
import matplotlib as plt
import numpy as np
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

#import data 
data = pd.read_csv("taxi_clean_med.csv")

#display data
print(data.shape)
      
data.head()

(8050, 19)


,trip_distance,fare_amount,winter,spring,summer,fall,PULongitude,PULatitude,DOLongitude,DOLatitude,pickup_datetime,dropoff_datetime,ride_duration,Early morning,Morning,Afternoon,Night,Holiday Proximity,label
0,1.1,7.0,0,1,0,0,-73.984176,40.759845,-73.972145,40.756816,2019-04-29 07:55:30,2019-04-29 08:03:39,0 days 00:08:09.000000000,0,1,0,0,0,B
1,2.6,12.0,0,0,1,0,-73.992455,40.748476,-74.008386,40.735248,2019-08-31 14:26:37,2019-08-31 14:42:27,0 days 00:15:50.000000000,0,0,1,0,0,D
2,1.2,5.5,0,0,1,0,-73.965174,40.756589,-73.951208,40.778496,2019-07-07 17:53:54,2019-07-07 17:57:55,0 days 00:04:01.000000000,0,0,1,0,0,B
3,1.9,10.5,0,1,0,0,-73.987973,40.775770,-73.978367,40.764425,2019-04-30 14:45:33,2019-04-30 14:58:43,0 days 00:13:10.000000000,0,0,1,0,0,D
4,1.0,5.5,0,0,0,1,-73.985214,40.727944,-73.976942,40.747654,2019-09-15 01:52:45,2019-09-15 01:57:18,0 days 00:04:33.000000000,1,0,0,0,0,B


In [12]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['pickup_datetime'] = pd.to_numeric(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])
data['dropoff_datetime'] = pd.to_numeric(data['dropoff_datetime'])

data['ride_duration'] = data['dropoff_datetime'] - data['pickup_datetime']

data.fillna(0, inplace= True)
print (data.dtypes)
data.head()

trip_distance        float64
fare_amount          float64
winter                 int64
spring                 int64
summer                 int64
fall                   int64
PULongitude          float64
PULatitude           float64
DOLongitude          float64
DOLatitude           float64
pickup_datetime        int64
dropoff_datetime       int64
ride_duration          int64
Early morning          int64
Morning                int64
Afternoon              int64
Night                  int64
Holiday Proximity      int64
label                 object
dtype: object


,trip_distance,fare_amount,winter,spring,summer,fall,PULongitude,PULatitude,DOLongitude,DOLatitude,pickup_datetime,dropoff_datetime,ride_duration,Early morning,Morning,Afternoon,Night,Holiday Proximity,label
0,1.1,7.0,0,1,0,0,-73.984176,40.759845,-73.972145,40.756816,1556524530000000000,1556525019000000000,489000000000,0,1,0,0,0,B
1,2.6,12.0,0,0,1,0,-73.992455,40.748476,-74.008386,40.735248,1567261597000000000,1567262547000000000,950000000000,0,0,1,0,0,D
2,1.2,5.5,0,0,1,0,-73.965174,40.756589,-73.951208,40.778496,1562522034000000000,1562522275000000000,241000000000,0,0,1,0,0,B
3,1.9,10.5,0,1,0,0,-73.987973,40.775770,-73.978367,40.764425,1556635533000000000,1556636323000000000,790000000000,0,0,1,0,0,D
4,1.0,5.5,0,0,0,1,-73.985214,40.727944,-73.976942,40.747654,1568512365000000000,1568512638000000000,273000000000,1,0,0,0,0,B


## Random Forest

In [28]:
# Split dataset into training and test set
# Split dataset into training and test sets
y = data['label']
X = data.drop(columns=['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)


# Fit Random Forest classifier on training set
random_forest = RandomForestClassifier(n_estimators=200, max_depth=5, max_features= 4, random_state=42)

# Run Cross Validation loop and print accuracy
c_score = cross_val_score(random_forest, X_train, y_train, cv=5)
print('Accuracy of classifier: ', c_score.mean())

# Add Grid
new_grid = {'max_depth':[5,10,15,20], 'min_samples_leaf':[5,10,15,20], 'max_features':[5,10,15]}
new_tune = GridSearchCV(random_forest,param_grid=new_grid,scoring='accuracy',cv=5)

# Pass training data into clasifier
new_tune.fit(X_train, y_train)

# Classify Test data
y_pred = new_tune.predict(X_test)

print('\n Best value for each of the tested parameters:',new_tune.best_params_)
print('\n and accuracy of the model with these best values is',new_tune.best_score_)

# Print Confusion Matrix
print('\n',confusion_matrix(y_test,y_pred))

# Priny confusion matrix
print('\n',classification_report(y_test,y_pred))

# Print accuracy score
print('\n',accuracy_score(y_test, y_pred))

Accuracy of classifier:  0.8274113771318572

 Best value for each of the tested parameters: {'max_depth': 10, 'max_features': 15, 'min_samples_leaf': 5}

 and accuracy of the model with these best values is 1.0

 [[ 442    0    0    0    0    0    0    0    0    0]
 [   0 1588    0    0    0    0    0    0    0    0]
 [   0    0 1311    0    0    0    0    0    0    0]
 [   0    0    0  918    0    0    0    0    0    0]
 [   0    0    0    0  606    0    0    0    0    0]
 [   0    0    0    0    0  362    0    0    0    0]
 [   0    0    0    0    0    0  261    0    0    0]
 [   0    0    0    0    0    0    0  171    0    0]
 [   0    0    0    0    0    0    0    0  136    0]
 [   0    0    0    0    0    0    0    0    0  645]]

               precision    recall  f1-score   support

           A       1.00      1.00      1.00       442
           B       1.00      1.00      1.00      1588
           C       1.00      1.00      1.00      1311
           D       1.00      1.00    

## Decision Trees

In [30]:
# Fit a decision tree classifier on the training set
decision_tree = DecisionTreeClassifier(random_state=0)

# Run Cross Validation loop and print accuracy
c_score = cross_val_score(decision_tree, X_train, y_train, cv=5)
n_pred = cross_val_predict(decision_tree, X_train, y_train, cv=5)
print('Accuracy of classifier: ', c_score.mean())
print('\n Confusion Matrix \n',confusion_matrix(y_test,n_pred))

'''# Add Grid
new_grid = {'max_depth':[5,10,15,20], 'min_samples_leaf':[5,10,15,20], 'max_features':[5,10,15]}
new_tune1 = GridSearchCV(decision_tree,param_grid=new_grid,scoring='accuracy',cv=5)

# Pass training data to classifier
new_tune1.fit(X_train, y_train)

print('\n Best value for each of the tested parameters:',new_tune1.best_params_)
print('\n and accuracy of the model with these best values is',new_tune1.best_score_)

# Run test data and predict classes
y_pred1 = new_tune1.predict(X_test)

# Print confusion matrix 
print('\n Confusion Matrix \n',confusion_matrix(y_test,n_pred))

# Print classification report
print('\n Classification report \n',classification_report(y_test,n_pred))

# Print accuracy score
print('\n Accuracy is:',accuracy_score(y_test, n_pred))'''

Accuracy of classifier:  1.0


ValueError: Found input variables with inconsistent numbers of samples: [6440, 1610]